<a href="https://colab.research.google.com/github/YassmineBizid/Vehicle-Detection-and-Counting-with-YOLO/blob/main/VehicleDetectionandCountingwithYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Vehicles Detection and Counting with YOLOv4 on images***

**Importing libraries**



In [ ]:
!pip install google.colab

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

**Installing Darknet**

Darknet is an open-source framework for object recognition, which can be used to train deep neural networks such as YOLO for object detection.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

**Importing YOLOv4 files**

YoloV4 (You Only Look Once version 4) is a real-time object detection algorithm based on the Yolo family.

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show test.mp4 -i 0 -out_filename results.avi

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights -P dnn_model/

In [ ]:
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg -P dnn_model/

**Vehicles detection with YOLOv4**

In [ ]:
class VehicleDetector:
  def __init__(self):
    net = cv2.dnn.readNet("dnn_model/yolov4.weights" , "dnn_model/yolov4.cfg")
    self.model = cv2.dnn_DetectionModel(net)
    self.model.setInputParams(size=(832,832) , scale=1 / 255)
    self.classes_allowed = [2,3,5,7] #classes for vehicles

  def detect_vehicles(self ,img):
      vehicles_boxes = []
      class_ids , score , boxes = self.model.detect(img , nmsThreshold = 0.4)
      for class_id , score , box in zip(class_ids , score , boxes):
        if score < 0.5 :
          continue
        if class_id in self.classes_allowed:
          vehicles_boxes.append(box)
      return vehicles_boxes

**Showing input images**

In [ ]:
vd = VehicleDetector()
import glob
images_folder = glob.glob("/content/*.jpg")
for img_path in images_folder:
  img = cv2.imread(img_path)
  if img is not None:
    cv2_imshow(img)
  else:
    print("Error while reading the image")


**Vehicles Detection**

In [ ]:
import glob
images_folder = glob.glob("/content/*.jpg")
for img_path in images_folder:
  img = cv2.imread(img_path)
  vehicle_boxes = vd.detect_vehicles(img)
  for box in vehicle_boxes:
        x, y, w, h = box
        cv2.rectangle(img, (x, y), (x + w, y + h), (225, 0, 180), 3)
  #cv2_imshow(img)

**Vehicles Counting**

In [ ]:
import glob
images_folder = glob.glob("/content/*.jpg")
vehicles_folder_count = 0
for img_path in images_folder:
    img = cv2.imread(img_path)
    vehicle_boxes = vd.detect_vehicles(img)
    vehicle_count = len(vehicle_boxes)
    for box in vehicle_boxes:
        x, y, w, h = box
        color = (0, 255,0 )
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 3)
        cv2.putText(img, " Vehicles:" + str(vehicle_count), (20, 50), 0, 2, (255, 0, 0), 3)
    cv2_imshow(img)